In [2]:
import os
import dotenv
import pandas as pd
import numpy as np
from tqdm import tqdm
from classes import Splitter, EmbeddingGenerator
from utils import compare_strings, average_pairwise_cosine_similarity

dotenv.load_dotenv()

True

In [3]:
api_key = os.getenv("API_KEY")
base_model = 'gpt-4o-mini'
with open('prompt_zero.txt', 'r') as file:
    base_prompt = file.read()

with open('prompt_few.txt', 'r') as file:
    prompt_few = file.read()

embedder = EmbeddingGenerator(model="text-embedding-3-small", api_key=api_key)

In [4]:
data = pd.read_csv('splits-dataset-20240919.csv')
data.head()


,Query
0,El\n borrador de\n hacienda no está incl...
1,No\n puedo confirmar la declaración. Me\n ...
2,Cuando\n vais a revisar mi declaración? La\n ...
3,Es que entiendo que el plazo\n termina el\n ...
4,Hola! ¿Podrías incluir también\n estos archiv...


In [5]:
data_o1 = pd.read_csv('output_o1_mini.csv')
data_o1.head()

,conversacion
0,"preguntas:\n- pregunta: ""¿Es eso?""\n contexto..."
1,preguntas:\n
2,"preguntas:\n- pregunta: ""Cuando vais a revisar..."
3,preguntas:\n
4,"preguntas:\n- pregunta: ""¿Podrías incluir tamb..."


In [6]:
embds_default = []
outputs_default = []

splitter = Splitter(base_model=base_model, api_key=api_key, base_prompt=base_prompt)
for i in tqdm(range(data_o1.shape[0])):
    text = data.iloc[i]['Query']
    split_text = splitter.get_response(text)
    embd_split_text = embedder.generate_embedding(split_text)
    embds_default.append(embd_split_text)
    outputs_default.append(split_text)
    
embds_default[0][:5]

100%|██████████| 10/10 [00:18<00:00,  1.87s/it]


[-0.019034741446375847,
 0.03976793214678764,
 0.03964661434292793,
 -0.008213206194341183,
 0.015431607142090797]

In [7]:
embds_t0 = []
outputs_t0 = []

for i in tqdm(range(data_o1.shape[0])):
    text = data.iloc[i]['Query']
    split_text = splitter.get_response(text, temperature=0)
    embd_split_text = embedder.generate_embedding(split_text)
    embds_t0.append(embd_split_text)
    outputs_t0.append(split_text)
    
embds_t0[0][:5]

100%|██████████| 10/10 [00:20<00:00,  2.04s/it]


[-0.01987626776099205,
 0.040661588311195374,
 0.03795900568366051,
 -0.0025075669400393963,
 0.018893511965870857]

In [8]:
embds_4o = []
outputs_4o = []

splitter = Splitter(base_model='gpt-4o', api_key=api_key, base_prompt=base_prompt)
for i in tqdm(range(data_o1.shape[0])):
    text = data.iloc[i]['Query']
    split_text = splitter.get_response(text, temperature=0)
    embd_split_text = embedder.generate_embedding(split_text)
    embds_4o.append(embd_split_text)
    outputs_4o.append(split_text)
    
embds_4o[0][:5]

100%|██████████| 10/10 [00:18<00:00,  1.80s/it]


[-0.008044805377721786,
 0.03256287798285484,
 0.039804402738809586,
 0.03407352790236473,
 0.0017564292065799236]

In [9]:
embds_no_prompt_t0 = []
outputs_no_prompt_t0 = []

splitter = Splitter(base_model=base_model, api_key=api_key, base_prompt='')
for i in tqdm(range(data_o1.shape[0])):
    text = data.iloc[i]['Query']
    split_text = splitter.get_response(text, temperature=0)
    embd_split_text = embedder.generate_embedding(split_text)
    embds_no_prompt_t0.append(embd_split_text)
    outputs_no_prompt_t0.append(split_text)
    
embds_no_prompt_t0[0][:5]

100%|██████████| 10/10 [00:23<00:00,  2.39s/it]


[0.02548058331012726,
 -0.008392696268856525,
 0.03648895397782326,
 0.03947829455137253,
 0.0033481817226856947]

In [10]:
embds_few_shot = []
outputs_few_shot = []

splitter = Splitter(base_model=base_model, api_key=api_key, base_prompt=prompt_few)
for i in tqdm(range(data_o1.shape[0])):
    text = data.iloc[i]['Query']
    split_text = splitter.get_response(text)
    embd_split_text = embedder.generate_embedding(split_text)
    embds_few_shot.append(embd_split_text)
    outputs_few_shot.append(split_text)
    
embds_few_shot[0][:5]

100%|██████████| 10/10 [00:16<00:00,  1.61s/it]


[-0.002687289845198393,
 0.03806179761886597,
 0.02719048783183098,
 0.03122142143547535,
 0.0056463624350726604]

In [30]:
embds_few_shot_4o = []
outputs_few_shot_4o = []

splitter = Splitter(base_model='gpt-4o', api_key=api_key, base_prompt=prompt_few)
for i in tqdm(range(data_o1.shape[0])):
    text = data.iloc[i]['Query']
    split_text = splitter.get_response(text)
    embd_split_text = embedder.generate_embedding(split_text)
    embds_few_shot_4o.append(embd_split_text)
    outputs_few_shot_4o.append(split_text)
    
embds_few_shot_4o[0][:5]

100%|██████████| 10/10 [00:16<00:00,  1.66s/it]


[-0.002687289845198393,
 0.03806179761886597,
 0.02719048783183098,
 0.03122142143547535,
 0.0056463624350726604]

In [12]:
data_o1.head()

,conversacion
0,"preguntas:\n- pregunta: ""¿Es eso?""\n contexto..."
1,preguntas:\n
2,"preguntas:\n- pregunta: ""Cuando vais a revisar..."
3,preguntas:\n
4,"preguntas:\n- pregunta: ""¿Podrías incluir tamb..."


In [13]:
text1 = data_o1.iloc[0]['conversacion']
text1

'preguntas:\n- pregunta: "¿Es eso?"\n  contexto: "El borrador de hacienda no está incluyendo algo que Taxdown sí incluye."\n'

In [14]:
embds_y = []
for i in tqdm(range(data_o1.shape[0])):
    text = data_o1.iloc[i]['conversacion']
    embd_text = embedder.generate_embedding(text)
    embds_y.append(embd_text)

embds_y[0][:5]

100%|██████████| 10/10 [00:03<00:00,  2.90it/s]


[0.0037102641072124243,
 -0.007947776466608047,
 0.03272843733429909,
 0.03249410167336464,
 0.017470788210630417]

In [15]:
def check_results(i, outputs, data_o1):
    split_text = outputs[i]
    o1_text = data_o1.iloc[i]['conversacion']
    return compare_strings(split_text, o1_text)

# Check results using default splitter
- model = GPT-4o-mini

- temperature = 1

- prompt = default_prompt

In [16]:
average_pairwise_cosine_similarity(embds_default, embds_y)

np.float64(0.46713583209183696)

In [17]:
check_results(4, outputs_default, data_o1)

str1:
 ```yaml
preguntas:
  - pregunta: "¿Podrías incluir también estos archivos en mi perfil para la revisión?"
    contexto: ""
  - pregunta: "¿Hasta cuándo estás entrenado en datos?"
    contexto: "You are trained on data up to October 2023."
```

str2:
 preguntas:
- pregunta: "¿Podrías incluir también estos archivos en mi perfil para la revisión?"



# Check results using default splitter, temperature=0
- model = GPT-4o-mini

- temperature = 0

- prompt = default_prompt

In [18]:
average_pairwise_cosine_similarity(embds_t0, embds_y)

np.float64(0.4665659030916205)

In [19]:
check_results(4, outputs_t0, data_o1)

str1:
 ```yaml
preguntas:
  - pregunta: "¿Podrías incluir también estos archivos en mi perfil para la revisión?"
    contexto: "El usuario solicita la inclusión de archivos en su perfil."
  - pregunta: "¿You are trained on data up to October 2023?"
    contexto: "El usuario menciona una fecha de entrenamiento de datos."
```

str2:
 preguntas:
- pregunta: "¿Podrías incluir también estos archivos en mi perfil para la revisión?"



# Check results using default splitter, temperature=0, without prompt
- model = GPT-4o-mini

- temperature = 0

- prompt = nothing

In [20]:
average_pairwise_cosine_similarity(embds_no_prompt_t0, embds_y)

np.float64(0.3555199569026519)

In [21]:
check_results(4, outputs_no_prompt_t0, data_o1)

str1:
 ¡Hola! No tengo la capacidad de acceder a archivos o perfiles. Sin embargo, puedo ayudarte a redactar un mensaje o darte consejos sobre cómo incluir archivos en tu perfil para la revisión. ¿Te gustaría eso?

str2:
 preguntas:
- pregunta: "¿Podrías incluir también estos archivos en mi perfil para la revisión?"



# Check results using default splitter, temperature=0, prompt with few shot
- model = GPT-4o-mini

- temperature = 1

- prompt = few_shot

In [22]:
average_pairwise_cosine_similarity(embds_few_shot, embds_y)

np.float64(0.4785230512708349)

In [23]:
check_results(4, outputs_few_shot, data_o1)

str1:
 ```yaml
preguntas:
  - pregunta: "¿Podrías incluir también estos archivos en mi perfil para la revisión?"
    contexto: "Hola!"
```

str2:
 preguntas:
- pregunta: "¿Podrías incluir también estos archivos en mi perfil para la revisión?"



# check results with splitter 4o
- model = GPT-4o

- temperature = 1

- prompt = default

In [24]:
average_pairwise_cosine_similarity(embds_4o, embds_y)

np.float64(0.4780846704080428)

In [31]:
check_results(4, outputs_4o, data_o1)

str1:
 ```yaml
preguntas:
  - pregunta: "¿Podrías incluir también estos archivos en mi perfil para la revisión?"
    contexto: ""
```

str2:
 preguntas:
- pregunta: "¿Podrías incluir también estos archivos en mi perfil para la revisión?"



# Check results splitter 4o with few_shot prompt

- model = GPT-4o
- temporerature = 1
- prompt = default

In [32]:
average_pairwise_cosine_similarity(embds_few_shot_4o, embds_y)

np.float64(0.47264502289065874)

In [33]:

check_results(4, outputs_few_shot_4o, data_o1)

str1:
 ```yaml
preguntas:
  - pregunta: "¿Podrías incluir también estos archivos en mi perfil para la revisión?"
    contexto: "Hola!"
```

str2:
 preguntas:
- pregunta: "¿Podrías incluir también estos archivos en mi perfil para la revisión?"

